<center>

<h1> Cuts and Negation </h1>
<h2> CS3100 Fall 2019 </h2>
</center>

## Review

### Previously

* Generate and Test: A design pattern for logic programming

### This lecture

* Cuts
  + A mechanism for pruning Prolog search trees
  + Red and Green cuts

## Evaluator

Consider a simple evaluator for arithmetic expressions.

In [ ]:
eval(plus(A,B),C) :- eval(A,VA), eval(B,VB), C is VA + VB.
eval(mult(A,B),C) :- eval(A,VA), eval(B,VB), C is VA * VB.
eval(A,A).

## Evaluator

What is the result of evaluating `1 + (4 * 5)`?

In [ ]:
?- eval(plus(1,mult(4,5)),X) {1}.

In [ ]:
?- eval(plus(1,mult(4,5)),X) {2}.

Trace `eval(plus(1,mult(4,5)),X)` by hand.

## Fixing the evaluator - with wrapper

Wrap the values in a function `value`.

In [ ]:
eval2(plus(A,B),C) :- eval2(A,VA), eval2(B,VB), C is VA + VB.
eval2(mult(A,B),C) :- eval2(A,VA), eval2(B,VB), C is VA * VB.
eval2(value(A),A).

In [ ]:
?- eval2(plus(value(1),mult(value(4),value(5))),X).

## Fixing the evaluator - with cut

* The **cut** (!) is an extra-logical (outside pure logic) operator that prunes the search trees. 
* When the evaluation cross a cut, it prunes
  + All the subsequent possible branches in the parent.
  + All the subsequent possible branches in the preceeding sub-goals.

In [ ]:
eval3(plus(A,B),C) :- !, eval3(A,VA), eval3(B,VB), C is VA + VB.
eval3(mult(A,B),C) :- !, eval3(A,VA), eval3(B,VB), C is VA * VB.
eval3(A,A).

In [ ]:
?- eval3(plus(1,mult(4,5)),X).

## Cut behaviour

In [1]:
p(a).
p(b).
r(c).
q(X) :- !, p(X).
q(X) :- r(X).

Added 5 clauses(s).

In [2]:
?- q(X).

X = a ;
X = b .

## Quiz

What does `split/3` do?

```prolog
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, split(T,L,R).
```

It splits the given list into elements less than 5 and greater than or equal to 5.

## Split

In [3]:
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, split(T,L,R).

Added 3 clauses(s).

In [4]:
?- split([1,2,3,4,5,6,7,8,9],L,R).

R = [ 5, 6, 7, 8, 9 ], L = [ 1, 2, 3, 4 ] .

* Observe that the last two cases are mutually exclusive. 
  + But Prolog still searches through the third rule, if second rule was successfully matched.

## Split with cut

In [5]:
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, !, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, !, split(T,L,R).

Added 2 clauses(s).

* The second `!` is unnecessary as there are no further choices.
  + In fact, the predicate `H >= 5` is unnecessary since the only way to end up here is if the first rule failed.
  + But better to leave it there for readability.
* Recommendation:
  + Use cut to optimise execution, but retain predicates which help readability. 

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$.
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$. **✓**
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,!,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$.
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Quiz

What is the logical meaning of these clauses?

```prolog
p :- a,!,b.
p :- c.
```

1. $p \leftrightarrow (a \wedge b) \vee c$.
2. $p \leftrightarrow a \wedge b \wedge c$.
3. $p \leftrightarrow (a \wedge b) \vee (\neg a \wedge c)$.  **✓**
4. $p \leftrightarrow a \wedge (b \vee c)$.

## Red and Green cuts

```prolog
p :- a,!,b.
p :- c.
```

Since the cut above changes the logical meaning of the program, it is known as **Red cut**.

```prolog
split([],[],[]).
split([H|T],[H|L],R) :- H < 5, !, split(T,L,R).
split([H|T],L,[H|R]) :- H >= 5, split(T,L,R).
```

The cut in split does not change the logical meaning of the program. Hence, it is called **Green cut**.

## Remove Stutter

In [ ]:
remove_stutter([],[]).
remove_stutter([H],[H]).
remove_stutter([H,H|T],L) :- !, remove_stutter([H|T],L).
remove_stutter([X,Y|T],[X|L]) :- remove_stutter([Y|T],L).

In [ ]:
?- remove_stutter([1,1,2,2,2,3,4,1,1],X).

## Remove Stutter

Can be equivalently written as:

In [ ]:
remove_stutter2([],[]).
remove_stutter2([H],[H]).
remove_stutter2([H,H|T],L) :- remove_stutter2([H|T],L).
remove_stutter2([X,Y|T],[X|L]) :- not(X=Y), remove_stutter2([Y|T],L).

In [ ]:
?- remove_stutter2([1,1,2,2,2,3,4,1,1],X).

## Negation

* What is the relationship between cut and negation?
* With the use of negation, the clause is not longer a definite clause. What's going on here?

## Quiz

What does this do?

```prolog
a :- !, 1=2.
?- a.
```

1. successfully unfies 1 with 2
2. throws an exception
3. loops indefinitely
4. fails

## Quiz

What does this do?

```prolog
a :- !, 1=2.
?- a.
```

1. successfully unfies 1 with 2
2. throws an exception
3. loops indefinitely
4. fails **✓**


* You can give a better name for `a/2`: `fail/0`.
  + Prolog has an built-in predicate `fail/0`, which always fails. 

## Quiz

What does this do?

```prolog
a(A,A) :- !,fail.
a(_,_).
```

1. unifies the two arguments
2. succeeds if the arguments unify
3. succeeds if the arguments don't unify
4. always fails

## Quiz

What does this do?

```prolog
a(A,A) :- !,fail.
a(_,_).
```

1. unifies the two arguments
2. succeeds if the arguments unify
3. succeeds if the arguments don't unify **✓**
4. always fails

You can give a better name for `a/2`: `is_different/2`.

## Failure on unification

In [ ]:
is_different(A,A) :- !,fail.
is_different(_,_).

In [ ]:
?- is_different(m,n).

## Behaviour of fail and is_different

* Clauses such as 'fail' and 'isDifferent' can cause us to backtrack in unusual ways.
* This will undo any variable unifications along the way.

## Negation by failure

We can now implement `not` using negation-by-failure.

```prolog
not(A) :- A,!,fail.
not(_).
```

In [ ]:
?- not(1=2).

* `not/2` is a built-in in Prolog.
* You may also write `not(A)` as `\+A`. 

## Quiz

What sort of a cut is this?

```prolog
not(A) :- A,!,fail.
not(_).
```

1. red.
2. amber.
3. green.

## Quiz

What sort of a cut is this?

```prolog
not(A) :- A,!,fail.
not(_).
```

1. red. **✓**
2. amber.
3. green.

If the cut were not there, then the first rule would always fail, but the second rule will always succeed.

## Closed world assumption

Everything that is true in the "world" is stated (or can be derived from) the clauses in the program. 

`not` is based on the closed world assumption.

`not(A)` holds if it cannot be shown from the given clauses that `A` holds.

## Example: Buying a phone

In [ ]:
goodPhone(iphone11pro).
goodPhone(oneplus7tpro).
expensive(iphone11pro).

bargain(X) :- goodPhone(X), not(expensive(X)).

In [ ]:
?- bargain(X).

## Simple mistake with negation

In [ ]:
goodPhone2(iphone11pro).
goodPhone2(oneplus7tpro).
expensive2(iphone11pro).

bargain2(X) :- not(expensive2(X)), goodPhone2(X).

In [ ]:
?- bargain2(X).

## What went wrong?

Trace through:

```prolog
goodPhone2(iphone11pro).
goodPhone2(oneplus7tpro).
expensive2(iphone11pro).

not(A) :- A,!,fail.
not(_).

bargain2(X) :- not(expensive2(X)), goodPhone2(X).

?- bargain2(X).
```

## When using negation remember the quantifier

Our negation is not a logical one.

`expensive2(X)` is $\exists x.expensive2(x)$.

Our `not(expensive2(X)` is $\neg (\exists x.expensive2(x)) \equiv \forall x.\neg expensive2(x))$

Hence, the rule

```prolog
bargain2(X) :- not(expensive2(X)), goodPhone2(X).
```

will only succeed if there are no expensive restaurants, which is not our intention.

* **Recommendation**: Use `not(T)` only when `T` is ground. 
  + This was the case in the first example.

<center>

<h1> Fin. </h1>
</center>